In [ ]:
# Download files

import requests, zipfile, io

files_url = "https://ideami.com/llm_align"
print("Downloading files using Python")
response = requests.get(files_url)
zipfile.ZipFile(io.BytesIO(response.content)).extractall(".")